In [ ]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
from collections import Counter
pd.options.mode.chained_assignment = None 
pd.set_option('display.max_columns', None)
df = pd.read_parquet('scontrini.parquet',engine='pyarrow')
df = df[(~df.reparto.isin(["95","80","85"]))&(df.tipo=="")]
df

In [ ]:
def hot_encode(x):
    if(x<= 0):
        return False
    if(x> 0):
        return True

Ottengo gli acquisti fatti solo da carte fedeltà

In [ ]:
transzioniCarte = df[['cassa','transazione','data_reg','descrizione','qta_riga','carta']]
transzioniCarte['carta']=transzioniCarte['carta'].str.strip()
transzioniCarte=transzioniCarte[transzioniCarte.carta!=""]
transzioniCarte

Creo il basket delle spese delle carte fedeltà

In [ ]:
speseCarte=transzioniCarte.groupby(['carta','cassa','transazione','data_reg'])\
    .count()\
    .reset_index()\
    .groupby(['carta'])\
    .count()\
    .drop(['cassa', 'transazione','data_reg','descrizione'], axis=1)\
    .rename(columns={"qta_riga": "spese"})\
    .reset_index()

#carteFiltrate=speseCarte[(speseCarte.spese>=10)&(speseCarte.spese<=180)].sort_values(by=['spese'],ascending=False)
#transazioniFiltrate = transzioniCarte[transzioniCarte.carta.isin(carteFiltrate.carta.values)]
speseCarte.sort_values(by="spese",ascending=False)

In [ ]:
basket=transzioniCarte.groupby(['cassa','transazione','data_reg','carta','descrizione']) \
    ['qta_riga']\
    .sum()\
    .unstack()\
    .reset_index()\
    .fillna(0) \
    .drop(['cassa', 'transazione'], axis=1)\
    .set_index(['carta', 'data_reg'])\
    .applymap(hot_encode)

basket

assosiation rules su tutte le carte di clienti 'veri'

In [ ]:
frq_items_all_cards = apriori(basket, min_support = 0.007, use_colnames = True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(frq_items_all_cards.sort_values(by=['support'],ascending=False))

In [ ]:
rules = association_rules(frq_items_all_cards, metric ="confidence", min_threshold = 0.01)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(rules.sort_values(by=['confidence'], ascending=False))

ottengo i 5 itemset più frequenti per ogni carta

In [ ]:
frq_items_cards={}
for card in carteFiltrate.carta:
    support=0.3
    nItems=0
    while(nItems<5):
        frq_items_card = apriori(basket[basket.index.get_level_values(0)==card], min_support = support, use_colnames = True).sort_values(by=['support'], ascending=False).head()
        nItems=len(frq_items_card)
        support-=0.01
    frq_items_cards[card]=frq_items_card

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_colwidth', 100):
    for card in frq_items_cards.keys():
        print("\n" +"="*50)
        print(card +"\t TRANSAZIONI: "+str(carteFiltrate[carteFiltrate.carta==card].spese.item()))
        print("-"*50)
        print(frq_items_cards[card])